In [1]:
import tensorflow as tf
import pandas as pd
import random

# Building Data Example

In [2]:
# Load Data
col_names = pd.read_pickle("../..//Data/PickleFiles/2_new_features/orig_and_eng.p").columns
xdata = pd.read_pickle("../..//Data/PickleFiles/3_labels_cv/training_data_cv2.p")[col_names]
ydata_orig = pd.read_pickle("../..//Data/PickleFiles/3_labels_cv/training_data_cv2labelweights-multiclass.p")['__label__']
# One hot encode the labels
ydata=pd.DataFrame(index=ydata_orig.index, columns=[0, 1, 2, 3, 4, 5], data=0)
for index in ydata.index:
    category_number = ydata_orig[index]
    ydata.loc[index, category_number] = 1

In [46]:
sess = tf.InteractiveSession()

# Model parameters
num_input_vars = 121
num_output_vars = 6

# Create the model
# Input
x = tf.placeholder(tf.float32, [None, num_input_vars])
# Output 
y_ = tf.placeholder(tf.float32, [None, num_output_vars])

# weights
W = tf.Variable(tf.zeros([num_input_vars, num_output_vars]))
# Biases
b = tf.Variable(tf.zeros([num_output_vars]))

# Initialize variables
sess.run(tf.global_variables_initializer())

# Implement regression mode
y = tf.matmul(x, W) + b
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

# Train
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [47]:
for i in range(1000):
    batch = []
    indices = random.sample(list(ydata.index.values), 100)
    batch.append(xdata.loc[indices, :])
    batch.append(ydata.loc[indices, :])
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [54]:
# Load the test data
xtest = pd.read_pickle("../..//Data/PickleFiles/3_labels_cv/testing_data_cv2.p")[col_names]
ytest_orig = pd.read_pickle("../..//Data/PickleFiles/3_labels_cv/testing_data_cv2labelweights-multiclass.p")['__label__']
ytest=pd.DataFrame(index=ytest_orig.index, columns=[0, 1, 2, 3, 4, 5], data=0)
# One hot encode the test data
for index in ytest.index:
    category_number = ytest_orig[index]
    ytest.loc[index, category_number] = 1

In [55]:
# Calculate the accuracy rate on the test data

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: xtest, y_: ytest}))

0.984551
